# GCC118 - Programação Matemática
## Universidade Federal de Lavras
### Problema 1
#### Aluno: Eduardo Cesar Cauduro Coelho
#### Matrícula: 202310175
#### Aluno: Felipe Geraldo de Oliveira
#### Matrícula: 202310174

## Instalação e importação da biblioteca PuLP


In [12]:
!pip install pulp
import pulp as pl

## Declaração dos parâmetros

* $A$: conjunto de aviões.     
* $P_a \in \mathbb{R}_+$: preço de cada avião $a \in A$.      
* $PI_a \in \mathbb{Z}_+$: quantidade pilotos aptos para pilotar cada avião $a \in A$.
* $R_a \in \mathbb{R}_+$: total de receita por avião $a \in A$.  
* $M_a \in \mathbb{R}_+$: fração de esforço de manutenção de cada cada avião $a \in A$ em comparação ao avião 'Boeing 717'.
* $v \in \mathbb{R}_+$: verba máxima para aquisição de aviões
* $l \in \mathbb{Z}_+$: limite de aviões 'Boeing 717' que as oficinas de manutenção suportam


In [13]:
avioes = ["BOEING 717", "BOEING 737-500", "MD-11"]

avioes_preco = {
    'BOEING 717': 5.1,
    'BOEING 737-500': 3.6,
    'MD-11': 6.8
}

pilotos_aptos = {
    'BOEING 717': 30,
    'BOEING 737-500': 20,
    'MD-11': 10
}

receita = {
    'BOEING 717': 330,
    'BOEING 737-500': 300,
    'MD-11': 420
}

avioes_manutencao = {
    'BOEING 717': 1.0,
    'BOEING 737-500': (3/4),
    'MD-11': (5/3)
}

verba_maxima = 220

limite_manutencao = 40

## Declaração do objeto que representa o modelo matemático

In [14]:
model = pl.LpProblem("receita_avioes", pl.LpMaximize)

## Variável de decisão

* $q_a \in \mathbb{Z}_+$: quantidade de aviões para cada $a \in A$.


In [15]:
avioes_var = pl.LpVariable.dicts("avioes", [aviao for aviao in avioes], lowBound=0, cat='Integer')

## Função objetivo

* Maximização das receitas dos aviões: \begin{equation}\max \sum \limits_{a \in A}(R_a \times q_a)\end{equation}

In [16]:
model += pl.lpSum([receita[aviao] * avioes_var[aviao] for aviao in avioes])

## Restrições


* A verba gasta na compra dos aviões é limitada superiomente por 220 (em milhões): \begin{equation}\sum \limits_{a \in A}(P_a \times q_a) ≤ v \end{equation}

In [17]:
model += pl.lpSum([avioes_preco[aviao] * avioes_var[aviao] for aviao in avioes]) <= verba_maxima

* O número de aviões comprados não pode ultrapassar a capacidade de manutenção das oficinas: \begin{equation}\sum \limits_{a \in A}(M_a \times q_a) ≤ l \end{equation}

In [18]:
model += pl.lpSum([avioes_var[aviao] * avioes_manutencao[aviao] for aviao in avioes]) <= limite_manutencao

* O número de aviões comprados não pode ultrapassar a quantidade disponível de pilotos para pilotá-los.
  
* Como cada avião necessita de dois pilotos para pilotá-lo: \begin{equation}\sum \limits_{a \in A}{q_a} ≤ \frac{1}{2}\times\sum \limits_{a \in A}{PI_a} \end{equation}
* O número de aviões de um modelo específico não deve ultrapassar a quantidade de pilotos disponíveis para pilotar este determinado modelo.
  
* Para o avião 'MD-11': \begin{equation}q_{md} \le \frac{PI_{md}}{2}\end{equation} onde $md$ representa o avião 'MD-11'

* Para o dois aviões restantes, os pilotos de aviões 'MD-11' também são capazes de pilotá-lo: \begin{equation}q_{md} + q_{a*} \le \frac{PI_{md}}{2} +\frac{PI_{a*}}{2}\end{equation} onde $a* \in A-\{md\}$

In [19]:

model += pl.lpSum([avioes_var[aviao] for aviao in avioes]) <= pl.lpSum([pilotos_aptos[aviao] for aviao in avioes])/2

for aviao in avioes:
    if aviao == 'MD-11':
        model += avioes_var[aviao] <= pilotos_aptos[aviao]/2
    else:
        model += avioes_var[aviao] + avioes_var["MD-11"]<= pilotos_aptos[aviao]/2 + pilotos_aptos["MD-11"]/2

## Apresentação do modelo

In [20]:
model

receita_avioes:
MAXIMIZE
330*avioes_BOEING_717 + 300*avioes_BOEING_737_500 + 420*avioes_MD_11 + 0
SUBJECT TO
_C1: 5.1 avioes_BOEING_717 + 3.6 avioes_BOEING_737_500 + 6.8 avioes_MD_11
 <= 220

_C2: avioes_BOEING_717 + 0.75 avioes_BOEING_737_500
 + 1.66666666667 avioes_MD_11 <= 40

_C3: avioes_BOEING_717 + avioes_BOEING_737_500 + avioes_MD_11 <= 30

_C4: avioes_BOEING_717 + avioes_MD_11 <= 20

_C5: avioes_BOEING_737_500 + avioes_MD_11 <= 15

_C6: avioes_MD_11 <= 5

VARIABLES
0 <= avioes_BOEING_717 Integer
0 <= avioes_BOEING_737_500 Integer
0 <= avioes_MD_11 Integer

## Resolvendo o problema

In [21]:
status = model.solve()

## Impressão das soluções do problema

In [22]:
print('status: ', pl.LpStatus[status])
print('funcao objetivo: ', model.objective.value())
print('solucoes')
for aviao in avioes:
  print(aviao,": ", avioes_var[aviao].value(), sep='')

status:  Optimal
funcao objetivo:  10050.0
solucoes
BOEING 717: 15.0
BOEING 737-500: 10.0
MD-11: 5.0
